Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [64]:
pip install pyspark

## Build Spark Session

In [65]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [66]:
trainDF = spark.read.csv("train2.csv",header=True,inferSchema='True')

testDF = spark.read.csv("test2.csv",header=True,inferSchema='True')

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [67]:
trainDF

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [68]:
trainDF.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

**Show 5 rows.**

In [69]:
trainDF.show(5, truncate=False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

**Display schema for the dataset:**

In [70]:
trainDF.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [71]:
trainDF.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [72]:
trainDF.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [73]:
people_survived = trainDF.select('Survived').where(trainDF['Survived']==1).count()

people_not_survived = trainDF.select('Survived').where(trainDF['Survived']==0).count()

total = trainDF.select('Survived').count()

**Display your result:**

In [74]:
people_survived

342

In [75]:
people_not_survived

549

**Can you display your answer in ratio form?(Hint: Use UDF.)**






In [76]:
def Ratio_fn(x,total1):
    return x*100/total1

In [77]:
ratio_of_survived = Ratio_fn(people_survived,total)

ratio_of_not_survived = Ratio_fn(people_not_survived,total)

In [78]:
ratio_of_survived

38.38383838383838

In [79]:
ratio_of_not_survived

61.61616161616162

**Can you get the number of males and females?**


In [80]:
people_male = trainDF.select('sex').where(trainDF['sex']=='male').count()

people_female = trainDF.select('sex').where(trainDF['sex']=='female').count()

In [81]:
people_male

577

In [82]:
people_female

314

**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

In [83]:
## What is the average number of survivors of each gender?
from pyspark.sql import functions as F
avg_survisor = trainDF.select('sex', 'survived').where(trainDF['survived']==1).groupBy(trainDF['sex']).\
                            agg(F.count(trainDF['survived']).alias('count_each'))

avg_survisor_each_gender = avg_survisor.withColumn('Avg_survisor_for_gender', 
                                                           avg_survisor['count_each']/342)

avg_survisor_each_gender.show()

+------+----------+-----------------------+
|   sex|count_each|Avg_survisor_for_gender|
+------+----------+-----------------------+
|female|       233|     0.6812865497076024|
|  male|       109|    0.31871345029239767|
+------+----------+-----------------------+



In [84]:
(trainDF.select('Survived','sex')
 .where(trainDF['Survived'] == 1)
 .groupBy(trainDF['sex'])
 .agg(F.count(trainDF['survived']))
 .show()
)

+------+---------------+
|   sex|count(survived)|
+------+---------------+
|female|            233|
|  male|            109|
+------+---------------+



**Create temporary view PySpark:**

In [85]:
trainDF.createOrReplaceTempView("temp_view")

**How many people survived, and how many didn't survive? By SQL:**

In [86]:
spark.sql("select  count(survived) from temp_view where (survived == 1)" ) \
     .show(truncate=False)

+---------------+
|count(survived)|
+---------------+
|342            |
+---------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

In [87]:
spark.sql('SELECT sex, count(survived)/(select count(*) from temp_view where survived==1) FROM temp_view WHERE survived=1 group by sex').show()

+------+--------------------------------------------------------------------+
|   sex|(CAST(count(survived) AS DOUBLE) / CAST(scalarsubquery() AS DOUBLE))|
+------+--------------------------------------------------------------------+
|female|                                                  0.6812865497076024|
|  male|                                                 0.31871345029239767|
+------+--------------------------------------------------------------------+



**Display a ratio for p-class:**


In [88]:
spark.sql("SELECT pclass, count(pclass)/(select count(*) from temp_view) as count  FROM temp_view group by pclass ").show()

+------+-------------------+
|pclass|              count|
+------+-------------------+
|     1|0.24242424242424243|
|     3| 0.5510662177328844|
|     2|0.20650953984287318|
+------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [89]:
total_set = trainDF.union(testDF)
total_count = total_set.count()

**Display count:**

In [90]:
total_count

1329

**Temporary view PySpark:**

In [91]:
total_set.createOrReplaceTempView("total_view")


**Can you define the number of null values in each column?**


In [92]:
from pyspark.sql.functions import isnull, when, count
nulls_df = total_set.select([count(when(isnull(c), c)).alias(c) for c in total_set.columns])
nulls_df.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [93]:
nulls_df = total_set.select([count(when(isnull(c), c)).alias(c) for c in total_set.columns])
nulls_df.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Preprocessing 

**Can you show me the name column from your temporary table?**

In [94]:
total_set.select('name').show(truncate=False)

+-------------------------------------------------------+
|name                                                   |
+-------------------------------------------------------+
|Braund, Mr. Owen Harris                                |
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |
|Heikkinen, Miss. Laina                                 |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)           |
|Allen, Mr. William Henry                               |
|Moran, Mr. James                                       |
|McCarthy, Mr. Timothy J                                |
|Palsson, Master. Gosta Leonard                         |
|Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)      |
|Nasser, Mrs. Nicholas (Adele Achem)                    |
|Sandstrom, Miss. Marguerite Rut                        |
|Bonnell, Miss. Elizabeth                               |
|Saundercock, Mr. William Henry                         |
|Andersson, Mr. Anders Johan                            |
|Vestrom, Miss

**Run this code:**

In [222]:
combined = total_set.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

In [223]:
title_col = combined.select(combined['title']).groupBy(combined['title']).\
                        agg(count(combined['title']).alias('count_title'))
title_col = title_col.orderBy('count_title', ascending=False)
title_col.show()

+--------+-----------+
|   title|count_title|
+--------+-----------+
|      Mr|        786|
|    Miss|        257|
|     Mrs|        186|
|  Master|         56|
|      Dr|         11|
|     Rev|          9|
|     Col|          4|
|    Mlle|          4|
|   Major|          3|
|Jonkheer|          2|
|Countess|          2|
|     Sir|          2|
|    Lady|          2|
|    Capt|          2|
|     Don|          1|
|     Mme|          1|
|      Ms|          1|
+--------+-----------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [224]:
# rare_lst = title_col.collect()
# rare_dect = {x[0]:'rare' for x in rare_lst}

rare_dect = {'Capt': 'rare',
 'Col': 'rare',
 'Countess': 'rare',
 'Don': 'rare',
 'Dr': 'rare',
 'Jonkheer': 'rare',
 'Lady': 'rare',
 'Major': 'rare',
 'Master': 'Master',
 'Miss': 'Miss',
 'Mlle': 'rare',
 'Mme': 'rare',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Ms': 'rare',
 'Rev': 'rare',
 'Sir': 'rare'}

**Run the function:**

In [225]:
def impute_title(title):
  if title in rare_dect:
    return rare_dect[title]
  else:
    return title

**Apply the function on "Title" column using UDF:**

In [226]:
from pyspark.sql.functions import udf
impute_titleUDF = udf(lambda z:impute_title(z))   

combined = combined.withColumn("Title", impute_titleUDF(F.col("title")))

**Display "Title" from table and group by "Title" column:**

In [227]:
combined.groupBy('Title').count().show()

+------+-----+
| Title|count|
+------+-----+
|  rare|   44|
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
+------+-----+



## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

In [228]:
from pyspark.sql.functions import mean


mean_val = combined.select(mean(combined['age'])).collect()[0][0]
mean_val

30.079501879699244

**Fill missing age with age mean:**

In [229]:
combined = combined.na.fill(mean_val,subset=['age'])

In [230]:
combined.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




In [231]:
embarked_grouped = combined.groupBy('Embarked').count().sort('Embarked')

**Show groupped_Embarked:**

In [235]:
embarked_grouped.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|    null|    3|
|       C|  253|
|       Q|  111|
|       S|  962|
+--------+-----+



**Get the  top groupped_Embarked:** 

In [237]:
embarked_grouped.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|    null|    3|
|       C|  253|
|       Q|  111|
|       S|  962|
+--------+-----+



In [239]:
value_fill_embark = embarked_grouped.select('embarked', 'count').where(embarked_grouped['embarked'].isNotNull()).collect()[-1][0]
value_fill_embark

'S'

**Fill missing values with Top 'S' of grouped_Embarked:**

In [240]:
combined_new = combined.na.fill(value_fill_embark, subset=['embarked'])

In [241]:

combined_new.select('embarked').groupBy('embarked').count().orderBy('embarked', ascending=False).show()

+--------+-----+
|embarked|count|
+--------+-----+
|       S|  965|
|       Q|  111|
|       C|  253|
+--------+-----+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [248]:
from pyspark.sql.functions import substring

In [249]:
combined_new2 = combined_new.withColumn('cabin', substring(F.col('cabin'), 0, 1))

**Show the result:**

In [250]:
combined_new2.select('cabin').groupBy('cabin').count().orderBy('count').show()

+-----+-----+
|cabin|count|
+-----+-----+
|    T|    1|
|    G|    4|
|    F|   18|
|    A|   23|
|    E|   51|
|    D|   52|
|    B|   77|
|    C|   82|
| null| 1021|
+-----+-----+



**Create the temporary view:**

In [251]:
combined_new2.createOrReplaceTempView('final_view')

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

In [252]:
combined_new2.select('cabin').groupBy('cabin').count().orderBy('count', ascending=False).show()

+-----+-----+
|cabin|count|
+-----+-----+
| null| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**Fill missing values with "U":**

In [253]:
final_dataset = combined_new2.na.fill('U', subset=['cabin'])
final_dataset.select('cabin').groupBy('cabin').count().orderBy('count', ascending=False).show()

+-----+-----+
|cabin|count|
+-----+-----+
|    U| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

____________________________________________

**Use Pipline to fit and transform:**

In [254]:
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.feature import StringIndexer, OneHotEncoder

final_dataset.dtypes


[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('cabin', 'string'),
 ('Embarked', 'string'),
 ('Title', 'string')]

In [255]:
categoricalCols = [field for (field, dataType) in final_dataset.dtypes
                   if dataType == "string"]
categoricalCols

['Name', 'Sex', 'Ticket', 'cabin', 'Embarked', 'Title']

In [256]:
categoricalCols

['Name', 'Sex', 'Ticket', 'cabin', 'Embarked', 'Title']

In [257]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Name_Index',
 'Sex_Index',
 'Ticket_Index',
 'cabin_Index',
 'Embarked_Index',
 'Title_Index']

In [258]:
oheOutputCols = [x + "_OHE" for x in categoricalCols]
oheOutputCols

['Name_OHE', 'Sex_OHE', 'Ticket_OHE', 'cabin_OHE', 'Embarked_OHE', 'Title_OHE']

In [259]:
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

In [260]:
numericCols = [field for (field,dataType) in trainDF.dtypes \
              if dataType != 'string' and field not in ['Survived','PassengerId'] ]
numericCols

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [261]:
assemblerInputs = oheOutputCols + numericCols
assemblerInputs

['Name_OHE',
 'Sex_OHE',
 'Ticket_OHE',
 'cabin_OHE',
 'Embarked_OHE',
 'Title_OHE',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare']

In [262]:
vecAssembler = VectorAssembler(inputCols= assemblerInputs,outputCol='features')

In [279]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline


rf = RandomForestClassifier(featuresCol='features' , labelCol='Survived',maxBins=100,seed=42)
pipeline =Pipeline(stages = [stringIndexer,oheEncoder,vecAssembler,rf ])

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [280]:
x_train, X_test = final_dataset.randomSplit([.8,.2],seed=42)


**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [281]:
pipelineModel = pipeline.fit(x_train)

In [282]:
predDF = pipelineModel.transform(X_test)

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [283]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predDF)

accuracy

0.7692307692307693

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
